## Q1: We're going to simulate a glider using Newton-Euler dynamics (50 pts)

In [1]:
import Pkg; Pkg.activate(dirname(@__DIR__)); Pkg.instantiate()

  Activating project at `~/Documents/Robotics PhD/Third Semester/16-715 Advanced Robot Dynamics/hw2-Pfisch23`
    Updating registry at `~/.julia/registries/General.toml`
   Installed RelocatableFolders ───────────── v1.0.0
   Installed MutableArithmetics ───────────── v0.2.22
   Installed FiniteDifferences ────────────── v0.12.25
   Installed OffsetArrays ─────────────────── v1.12.8
   Installed StaticArrays ─────────────────── v1.5.9
   Installed DifferentialEquations ────────── v7.5.0
   Installed EnumX ────────────────────────── v1.0.3
   Installed RecipesPipeline ──────────────── v0.6.7
   Installed Sundials_jll ─────────────────── v5.2.1+0
   Installed DSP ──────────────────────────── v0.7.7
   Installed OpenBLAS32_jll ───────────────── v0.3.17+0
   Installed FileIO ───────────────────────── v1.16.0
   Installed MeshCat ──────────────────────── v0.13.2
   Installed BoundaryValueDiffEq ──────────── v2.9.0
   Installed SteadyStateDiffEq ────────────── v1.9.0
   Installed RandomNumbe

  [77dc65aa] + FunctionWrappersWrappers v0.1.1
  [de31a74c] + FunctionalCollections v0.5.0
  [46192b85] + GPUArraysCore v0.1.2
  [28b8d3ca] + GR v0.69.5
  [c145ed77] + GenericSchur v0.5.3
  [5c1252a2] + GeometryBasics v0.3.13
  [86223c79] + Graphs v1.7.4
  [42e2da0e] + Grisu v1.0.2
  [cd3eb016] + HTTP v0.9.17
  [9fb69e20] + Hiccup v0.2.2
  [3e5b6fbb] + HostCPUFeatures v0.1.8
  [e91730f6] + Hungarian v0.6.0
  [34004b35] + HypergeometricFunctions v0.3.11
  [615f187c] + IfElse v0.1.1
  [d25df0c9] + Inflate v0.1.3
  [83e8ac13] + IniFile v0.5.1
  [3587e190] + InverseFunctions v0.1.8
  [b6b21f68] + Ipopt v0.7.0
  [92d709cd] + IrrationalConstants v0.1.1
  [c8e1da08] + IterTools v1.4.0
  [42fd0dbc] + IterativeSolvers v0.9.2
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [033835bb] + JLD2 v0.4.25
  [1019f520] + JLFzf v0.1.5
  [692b3bcd] + JLLWrappers v1.4.1
  [97c1335a] + JSExpr v0.5.4
  [682c06a0] + JSON v0.21.3
  [7d188eb4] + JSONSchema v0.3.4
  [ccbc3e58] + JumpProcesses v9.2.1
  [ef3ab

  [30392449] + Pixman_jll v0.40.1+0
  [ea2cea3b] + Qt5Base_jll v5.15.3+1
  [f50d1b31] + Rmath_jll v0.3.0+0
  [fb77eaff] + Sundials_jll v5.2.1+0
  [a2964d1f] + Wayland_jll v1.19.0+0
  [2381bf8a] + Wayland_protocols_jll v1.25.0+0
  [02c8fc9c] + XML2_jll v2.9.14+0
  [aed1982a] + XSLT_jll v1.1.34+0
  [4f6342f7] + Xorg_libX11_jll v1.6.9+4
  [0c0b7dd1] + Xorg_libXau_jll v1.0.9+4
  [935fb764] + Xorg_libXcursor_jll v1.2.0+4
  [a3789734] + Xorg_libXdmcp_jll v1.1.3+4
  [1082639a] + Xorg_libXext_jll v1.3.4+4
  [d091e8ba] + Xorg_libXfixes_jll v5.0.3+4
  [a51aa0fd] + Xorg_libXi_jll v1.7.10+4
  [d1454406] + Xorg_libXinerama_jll v1.1.4+4
  [ec84b674] + Xorg_libXrandr_jll v1.5.2+4
  [ea2f1a96] + Xorg_libXrender_jll v0.9.10+4
  [14d82f49] + Xorg_libpthread_stubs_jll v0.1.0+3
  [c7cfdc94] + Xorg_libxcb_jll v1.13.0+3
  [cc61e674] + Xorg_libxkbfile_jll v1.1.0+4
  [12413925] + Xorg_xcb_util_image_jll v0.4.0+1
  [2def613f] + Xorg_xcb_util_jll v0.4.0+1
  [975044d2] + Xorg_xcb_util_keysyms_jll v0.4.0+1
  [0d4

In [1]:
using LinearAlgebra
using StaticArrays
using ForwardDiff
using Plots
using JLD2
using Test
using FileIO
plotlyjs()

const jld2path = joinpath(dirname(@__DIR__),"testfiles")
include(joinpath(jld2path,"test_functions.jl"))
test_dict = load(joinpath(jld2path,"q1.jld2"));

SyntaxError: invalid syntax (<ipython-input-1-d81eee49ff76>, line 1)

In [ ]:
#Some standard functions for dealing with rotation matrices and quaternions from the class notes

function hat(ω)
    return [0    -ω[3]  ω[2];
            ω[3]  0    -ω[1];
           -ω[2]  ω[1]  0   ]
end

function L(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I + hat(q[2:4])]
end

function R(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I - hat(q[2:4])]
end

const H = [zeros(1,3); I];

In [ ]:
# these are constants, don't change these

# NOTE: re-running this cell will result in "WARNING: redefinition of constant" warnings, don't worry about these

#Glider parameters

const g = 9.81; #Gravitational acceleration (m/s^2)
const ρ = 1.225; #Air density at 20C (kg/m^3)
const m = .484; #Mass of plane (kg)

# Inertia (kg*m^2)
const Jx  = 0.003922;
const Jy  = 0.015940;
const Jz  = 0.019340;
const Jxz = 0.000441;
const J = [Jx 0 Jxz;
     0  Jy  0;
     Jxz 0 Jz];

# Main Wing
const b = 86.4/100; #wing span (m)
const cr = 26/100; #root chord (m)
const ct = 15.2/100; #tip chord (m)
const cm = (ct + cr)/2; #mean wing chord (m)
const S = b*cm; #planform area of wing (m^2)
const Ra = b^2/S; #wing aspect ratio (dimensionless)
const Rt = ct/cr; #wing taper ratio (dimensionless)
const r_wing1 = [0; (b/6)*(1+2*Rt)/(1+Rt); 0]; #vector from CoM to wing center-of-pressure (m)
const r_wing2 = [0; -(b/6)*(1+2*Rt)/(1+Rt); 0];
const ϵ_ail = 0.45 #flap effectiveness (dimensionless)

# Elevator
const b_ele = 18.2/100; #elevator span (m)
const cr_ele = 15.2/100; #elevator root chord (m)
const ct_ele = 14/100; #elevator tip chord (m)
const cm_ele = (ct_ele + cr_ele)/2; #mean elevator chord (m)
const S_ele = b_ele*cm_ele; #planform area of elevator (m^2)
const Ra_ele = b_ele^2/S_ele; #elevator aspect ratio (dimensionless)
const r_ele = [-45/100; 0; 0]; #vector from CoM to elevator center-of-pressure (m)
const ϵ_ele = 0.8 #flap effectiveness (dimensionless)

# Rudder
const b_rud = 21.6/100; #rudder span (m)
const cr_rud = 20.4/100; #rudder root chord (m)
const ct_rud = 12.9/100; #rudder tip chord (m)
const cm_rud = (ct_rud + cr_rud)/2; #mean rudder chord (m)
const S_rud = b_rud*cm_rud; #planform area of rudder (m^2)
const Ra_rud = b_rud^2/S_rud; #rudder aspect ratio (dimensionless)
const r_rud = [-48/100; 0; -3/100] #vector from CoM to rudder center-of-pressure (m)
const ϵ_rud = 0.7 #flap effectiveness (dimensionless)

# Lift and drag polynomial coefficients from wind tunnel data
const Clcoef = [38.779513049043175; 0.0; 19.266141214863080; 0.0; -13.127972418509980; 0.0; 3.634063117174400; 0.0];
const Cdcoef = [3.607550808703421; 0.0; -4.489225907857385; 0.0; 3.480420330498847; 0.0; 0.063691497636087];

## 1A: glider dynamics (15 pts)

<div align="center">
    <figure>
    <img src="images/plane.png" style="width:500px">
        <figcaption align = "center">Glider body and axes of rotation. The <i>z</i>-axis points <b>down</b> in this frame. Image from <a href="https://www.google.com/url?sa=i&url=https%3A%2F%2Fpilotinstitute.com%2Feverything-about-ailerons%2F&psig=AOvVaw09rUJIWjsKuq3X8ve8-FxT&ust=1664897583842000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCMDc5f6wxPoCFQAAAAAdAAAAABAT">here</a>.</figcaption>
    </figure>
</div>

In [ ]:
#These functions calculate lift and drag coefficients as a function of angle-of-attack
#(or side-slip angle in the case of the rudder)

function Cl_wing(α)
    Cl = Clcoef'*(α.^((length(Clcoef)-1):-1:0))
end

function Cd_wing(α)
    Cd = Cdcoef'*(α.^((length(Cdcoef)-1):-1:0))
end

function Cl_ele(α)
    Cl = (pi/2)*Ra_ele*α
end

function Cd_ele(α)
    Cd = (pi/4)*Ra_ele*α*α
end

function Cl_rud(β)
    Cl = (pi/2)*Ra_rud*β
end

function Cd_rud(β)
    Cd = (pi/4)*Ra_rud*β*β
end


# TODO: fill these two functions out 
function aoa(v)
    #Calculate angle-of-attack from body-frame velocity
    at_a = atan(v[3],v[1])
    return at_a
end

function ss(v)
    #Calculate side-slip angle from body-frame velocity
    #This is similar to angle-of-attack, but for yaw rather than pitch
    sideslip = atan(v[2],v[1])
    return sideslip
end

ss (generic function with 1 method)

In [ ]:
using Test, JLD2, Random, FileIO
@testset "block 1" begin 
    α1 = deg2rad(4)
    β1 = deg2rad(23)
    v1 = [1;3;-4]
    g1 = [Cl_wing(α1);Cd_wing(α1);Cl_ele(α1);Cd_ele(α1);Cl_rud(β1);Cd_rud(β1);aoa(v1);ss(v1)]
    
    α2 = deg2rad(-3)
    β2 = deg2rad(-13)
    v2 = [-1;0.34;-5.6]
    g2 = [Cl_wing(α2);Cd_wing(α2);Cl_ele(α2);Cd_ele(α2);Cl_rud(β2);Cd_rud(β2);aoa(v2);ss(v2)]
    
    test_vec(g1,"g1",test_dict)
    test_vec(g2,"g2",test_dict)
end

Test Summary: | Pass  Total
block 1       |   16     16


Test.DefaultTestSet("block 1", Any[Test.DefaultTestSet("element #1 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #2 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #3 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #4 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #5 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #6 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #7 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #8 of g1", Any[], 1, false, false), Test.DefaultTestSet("element #1 of g2", Any[], 1, false, false), Test.DefaultTestSet("element #2 of g2", Any[], 1, false, false), Test.DefaultTestSet("element #3 of g2", Any[], 1, false, false), Test.DefaultTestSet("element #4 of g2", Any[], 1, false, false), Test.DefaultTestSet("element #5 of g2", Any[], 1, false, false), Test.DefaultTestSet("element #6 of g2", Any[], 1, false, false), Test.DefaultTestSet("element #7 of g2", Any[], 1, false

In [ ]:
#Plot Cl and Cd for the main wing
α = LinRange(-20*(pi/180),20*(pi/180),100)
cls = zeros(length(α))
cds = zeros(length(α))
for k = 1:length(α)
    cls[k] = Cl_wing(α[k])
    cds[k] = Cd_wing(α[k])
end
plot((180/pi).*α,cls,label = "Cl")
plot!((180/pi).*α,cds, label = "Cd")
xlabel!("Angle of Attack (degree)")

In [ ]:
# TODO: compute the aerodynamic forces and torques on the glider from each lifting surface

function aero_forces(v,ω,u;unit_testing = false)
    
    # inputs: 
    # v body frame velocity
    # ω body frame angular velocity
    # u controls [aileron, elevator, rudder]
    
    # outputs:
    # F total aerodynamic force vector in the body frame
    # τ total aerodynamic torque vector in the body frame
    
    
    # Template code
    v_wing1 = v + hat(ω)*r_wing1#Calculate the velocity of the right wing's center of pressure, accounting for the glider's angular velocity
    α_wing1 = aoa(v_wing1)
    α_eff_wing1 = α_wing1 - ϵ_ail*u[1] #Calculate effective angle of attack accounting for aileron defflection
    L_wing1 = Cl_wing(α_eff_wing1)*0.5*ρ*norm(v_wing1)^2*S#Calculate lift component in the wind frame
    D_wing1 = Cd_wing(α_eff_wing1)*0.5*ρ*norm(v_wing1)^2*S#Calculate drag component in the wind frame
    F_wing1 = αrotate(α_wing1,L_wing1,D_wing1) #rotate lift and drag into the glider's body frame
    τ_wing1 = cross(r_wing1,F_wing1)#Calculate torque about the glider's CoM due to lift and drag
    
    v_wing2 = v + hat(ω)*r_wing2 #Calculate the velocity of the left wing's center of pressure, accounting for the glider's angular velocity
    α_wing2 = aoa(v_wing2)
    α_eff_wing2 = α_wing2 + ϵ_ail*u[1]#Calculate effective angle of attack accounting for aileron defflection
    L_wing2 = Cl_wing(α_eff_wing2)*0.5*ρ*norm(v_wing2)^2*S#Calculate lift component in the wind frame
    D_wing2 = Cd_wing(α_eff_wing2)*0.5*ρ*norm(v_wing2)^2*S#Calculate drag component in the wind frame
    F_wing2 = αrotate(α_wing2,L_wing2,D_wing2) #rotate lift and drag into the glider's body frame
    τ_wing2 = cross(r_wing2,F_wing2)#Calculate torque about the glider's CoM due to lift and drag
    
    v_ele = v + hat(ω)*r_ele#Calculate the velocity of the elevator's center of pressure, accounting for the glider's angular velocity
    α_ele = aoa(v_ele)
    α_eff_ele = α_ele - ϵ_ele*u[2]#Calculate effective angle of attack accounting for elevator defflection
    L_ele = Cl_ele(α_eff_ele)*0.5*ρ*norm(v_ele)^2*S_ele #Calculate lift component in the wind frame !!!!NOTE: use S not S_ele!!!!
    D_ele = Cd_ele(α_eff_ele)*0.5*ρ*norm(v_ele)^2*S_ele#Calculate drag component in the wind frame !!!!NOTE: use S not S_ele!!!!
    F_ele = αrotate(α_ele,L_ele,D_ele) #rotate lift and drag into the glider's body frame
    τ_ele = cross(r_ele,F_ele)#Calculate torque about the glider's CoM due to lift and drag
    
    v_rud = v + hat(ω)*r_rud#Calculate the velocity of the rudder's center of pressure, accounting for the glider's angular velocity
    β_rud = ss(v_rud) #side slip angle of rudder
    β_eff_rud = β_rud + ϵ_rud*u[3]#Calculate effective side slip accounting for rudder defflection
    L_rud = Cl_rud(β_eff_rud)*0.5*ρ*norm(v_rud)^2*S_rud#Calculate lift component in the wind frame !!!!NOTE: use S not S_rud!!!!
    D_rud = Cd_rud(β_eff_rud)*0.5*ρ*norm(v_rud)^2*S_rud#Calculate drag component in the wind frame !!!!NOTE: use S not S_rud!!!!
    F_rud = βrotate(β_rud,L_rud,D_rud) #rotate lift and drag into the glider's body frame
    τ_rud = cross(r_rud,F_rud)#Calculate torque about the glider's CoM due to lift and drag
    
    # unit tests 
    if unit_testing         
        test_vec(v_wing1,"v_wing1", aeroforce_sol)
        test_vec(α_wing1,"α_wing1", aeroforce_sol)
        test_vec(α_eff_wing1,"α_eff_wing1", aeroforce_sol)
        test_vec(L_wing1 ,"L_wing1", aeroforce_sol)
        test_vec(D_wing1 ,"D_wing1", aeroforce_sol)
        test_vec(F_wing1,"F_wing1", aeroforce_sol)
        test_vec(τ_wing1,"τ_wing1", aeroforce_sol)
        
        test_vec(v_wing2,"v_wing2", aeroforce_sol)
        test_vec(α_wing2,"α_wing2", aeroforce_sol)
        test_vec(α_eff_wing2,"α_eff_wing2", aeroforce_sol)
        test_vec(L_wing2 ,"L_wing2", aeroforce_sol)
        test_vec(D_wing2 ,"D_wing2", aeroforce_sol)
        test_vec(F_wing2,"F_wing2", aeroforce_sol)
        test_vec(τ_wing2,"τ_wing2", aeroforce_sol)
        
        test_vec(v_ele,"v_ele", aeroforce_sol)
        test_vec(α_ele,"α_ele", aeroforce_sol)
        test_vec(α_eff_ele,"α_eff_ele", aeroforce_sol)
        test_vec(L_ele ,"L_ele", aeroforce_sol)
        test_vec(D_ele ,"D_ele", aeroforce_sol)
        test_vec(F_ele,"F_ele", aeroforce_sol)
        test_vec(τ_ele,"τ_ele", aeroforce_sol)
        
        test_vec(v_rud,"v_rud", aeroforce_sol)
        test_vec(β_rud,"β_rud", aeroforce_sol)
        test_vec(β_eff_rud,"β_eff_rud", aeroforce_sol)
        test_vec(L_rud ,"L_rud", aeroforce_sol)
        test_vec(D_rud ,"D_rud", aeroforce_sol)
        test_vec(F_rud,"F_rud", aeroforce_sol)
        test_vec(τ_rud,"τ_rud", aeroforce_sol)    
    end
    
    F = F_wing1 + F_wing2 + F_ele + F_rud
    τ = τ_wing1 + τ_wing2 + τ_ele + τ_rud
    return F, τ 
end

function αrotate(α,L,D)
    #Rotate by angle of attack from wind frame into body frame
    F =[-D;0;-L]
    Q = [cos(α) 0 -sin(α)
    0 1 0
    sin(α) 0 cos(α)]
    F_rot = Q*F
    #display(F_rot)
    return F_rot
end

function βrotate(β,L,D)
    #Rotate by sideslip angle from wind frame into body frame
    F =[-D;-L;0]
    Q = [cos(β) -sin(β) 0
    sin(β) cos(β) 0
        0 0 1]
    F_rot = Q*F
    #display(F_rot)
    return F_rot
end

βrotate (generic function with 1 method)

In [ ]:
@testset "αrotate and βrotate" begin 
    α = 1.234
    β = -0.3
    L = 32
    D = -32
    
    R1_2 = αrotate(α,L,D)
    R2_2 = βrotate(β,L,D)
       
    @test size(R1_2) == (3,)
    @test size(R2_2) == (3,)
    test_vec(R1_2,"R1_2",test_dict)
    test_vec(R2_2,"R2_2",test_dict)
end

Test Summary:       | Pass  Total
αrotate and βrotate |    8      8


Test.DefaultTestSet("αrotate and βrotate", Any[Test.DefaultTestSet("element #1 of R1_2", Any[], 1, false, false), Test.DefaultTestSet("element #2 of R1_2", Any[], 1, false, false), Test.DefaultTestSet("element #3 of R1_2", Any[], 1, false, false), Test.DefaultTestSet("element #1 of R2_2", Any[], 1, false, false), Test.DefaultTestSet("element #2 of R2_2", Any[], 1, false, false), Test.DefaultTestSet("element #3 of R2_2", Any[], 1, false, false)], 2, false, false)

In [ ]:
aeroforce_sol = load(joinpath(jld2path,"aeroforce.jld2"));
@testset "aeroforces" begin 
    v = [10;-.2;.4];
    ω = [-.1;.3;0.2];
    u = [-1;0.4;-0.4]
    
    F1, τ1 = aero_forces(v,ω,u;unit_testing = true)
       
    test_vec(F1,"F1",test_dict)
    test_vec(τ1,"τ1",test_dict)
    
    v = [16;0.3;-.23];
    ω = [.1;-.15;0.02];
    u = [-0.26;-0.08;0.2]
    
    F2, τ2 = aero_forces(v,ω,u)
        
    test_vec(F2,"F2",test_dict)
    test_vec(τ2,"τ2",test_dict)
end

Test Summary: | Pass  Total
aeroforces    |   64     64


Test.DefaultTestSet("aeroforces", Any[Test.DefaultTestSet("element #1 of v_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #2 of v_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #3 of v_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #1 of α_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #1 of α_eff_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #1 of L_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #1 of D_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #1 of F_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #2 of F_wing1", Any[], 1, false, false), Test.DefaultTestSet("element #3 of F_wing1", Any[], 1, false, false)  …  Test.DefaultTestSet("element #3 of F1", Any[], 1, false, false), Test.DefaultTestSet("element #1 of τ1", Any[], 1, false, false), Test.DefaultTestSet("element #2 of τ1", Any[], 1, false, false), Test.DefaultTestSet("element #3 of τ1", Any[], 1, false, fal

In [ ]:
#Using the aerodynamic forces and torques, implement the Newton-Euler dynamics for the glider

function glider_dynamics_q(x,u)
    #Unpack state vector
    r = x[1:3] #N frame
    q = x[4:7] #B to N
    v = x[8:10] #B frame
    ω = x[11:13] #B frame
    F,τ = aero_forces(v,ω,u)
    Q = H'*L(q)*R(q)'*H
    ṙ = Q*v
    q̇ = 0.5*L(q)*H*ω
    v̇ = F/m - cross(ω,v) - Q'*[0,0,g]
    ω̇ = J\(τ - cross(ω,J*ω))
    return [ṙ;q̇;v̇;ω̇]
end

glider_dynamics_q (generic function with 1 method)

In [ ]:
@testset "glyder dynamics" begin 
    r0 = [-10.0; -0.2; 10.0]
    r = normalize([1;0.07;-0.2;0.21])
    θ = pi*0.99;
    q0 = [cos(θ/2);r*sin(θ/2)]
    v0 = [10.0; 0.1; 0]
    ω0 = [0.1;0.2;-0.3]
    x0 = [r0; q0; v0; ω0];
    u0 = [0.1;-0.3;0.24];
    
    xdot1 = glider_dynamics_q(x0,u0)
    
    test_vec(xdot1,"xdot1",test_dict)
end

Test Summary:   | Pass  Total
glyder dynamics |   13     13


Test.DefaultTestSet("glyder dynamics", Any[Test.DefaultTestSet("element #1 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #2 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #3 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #4 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #5 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #6 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #7 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #8 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #9 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #10 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #11 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #12 of xdot1", Any[], 1, false, false), Test.DefaultTestSet("element #13 of xdot1", Any[], 1, false, false)], 0, false, false)

## 1B: Find initial conditions for trim (15 pts)

In [ ]:
#Nominal Initial Conditions (leave this alone)

#note that the conventional body-frame coordinate system for airplanes is:
# x pointing out the nose
# y pointing out the right wing
# z pointing out the floor

r0 = [-10.0; 0; 10.0]
q0 = [cos(pi/2); sin(pi/2); 0; 0] #this rotates the airplane so it is right-side up
v0 = [10.0; 0; 0]
ω0 = zeros(3)
x0 = [r0; q0; v0; ω0];
u0 = zeros(3);

In [ ]:
#We're now going to find trim conditions for forward flight. This is similar to an equilibrium.
#We define trim as motion at constant velocity and constant attitude (v̇=0 and ω=0)
#In general, this will require non-zero pitch, vertical velocity, and elevator inputs

function trim_error(z)
    # input is: z = [pitch; velocity z-component; elevator]
    # output should be err = [v̇x, v̇z, ω̇y]
    
    # hint: use the input pitch, vz and elevator to update the initial conditions within this function. 
    # update the attitude by rotating the plane about the y axis
    
    pitch = z[1]
    vz = z[2]
    ele = z[3]
    cpitch = cos(pitch/2)
    spitch = sin(pitch/2)
    
    # apply your updates to r, q, v, ω, and u here 
    r = r0 + [0;0;0]
    q = L(q0)*[cpitch;0;spitch;0]
    v = v0 + [0;0;vz]
    ω = ω0 + [0;0;0]
    u = u0 + [0;ele;0]
    
    # assemble the state vector 
    x = [r;q;v;ω]
    
    # call the dynamics with this 
    xdot = glider_dynamics_q(x,u)
    
    # you will need to pull out the desired output from xdot 
    
    rdot = xdot[1:3] #N frame
    qdot = xdot[4:7] #B to N
    vdot = xdot[8:10] #B frame
    ωdot = xdot[11:13] #B frame
    
    v̇x, v̇z, ω̇y = vdot[1], vdot[3], ωdot[2]
    
    return [v̇x, v̇z, ω̇y]
end

trim_error (generic function with 1 method)

In [ ]:
@testset "trim_error" begin 
    
    z = [0.2;3.0;-0.5]
    
    err1 = trim_error(z)
        
    test_vec(err1,"err1",test_dict)
end

Test Summary: | Pass  Total
trim_error    |    3      3


Test.DefaultTestSet("trim_error", Any[Test.DefaultTestSet("element #1 of err1", Any[], 1, false, false), Test.DefaultTestSet("element #2 of err1", Any[], 1, false, false), Test.DefaultTestSet("element #3 of err1", Any[], 1, false, false)], 0, false, false)

In [ ]:
# TODO: Use Newton's method to solve for trim conditions, z
z = [0.0; 0.0; 0.0] #initial guess for z 

# Newton's method here:

for i = 1:50
    ∇f = ForwardDiff.jacobian(_x -> trim_error(_x), z)
    Δz = -∇f\trim_error(z)
    z += Δz
    if norm(Δz) < 10e-10
        return z
    end

end

3-element Vector{Float64}:
 -0.29529394440239354
  0.5678625279339364
  0.07090666435590914

In [ ]:
@testset "newtons method on trim" begin 
    @test norm(trim_error(z)) < 1e-10
end

Test Summary:          | Pass  Total
newtons method on trim |    1      1


Test.DefaultTestSet("newtons method on trim", Any[], 1, false, false)

In [ ]:
# TODO: update initial conditions (r0,q0,v0,ω0,x0,u0) with the (z) from the previous section
pitch = z[1]
vz = z[2]
ele = z[3]
cpitch = cos(pitch/2)
spitch = sin(pitch/2)
q0 = L(q0)*[cpitch;0;spitch;0] 
v0 = v0 + [0;0;vz]
x0 = [r0; q0; v0; ω0]
u0 = u0 + [0;ele;0] 



3-element Vector{Float64}:
 0.0
 0.07090666435590914
 0.0

In [ ]:
@testset "updated trim IC's" begin 
    @test isapprox(x0,[r0;q0;v0;ω0],atol = 1e-12)
    @test isapprox(0,u0[1],atol = 1e-12)
    @test isapprox(0,u0[3],atol = 1e-12)
    @test abs(u0[2]) > 0 
end

Test Summary:     | Pass  Total
updated trim IC's |    4      4


Test.DefaultTestSet("updated trim IC's", Any[], 4, false, false)

In [ ]:
# TODO: Simulate the glider from the given initial conditions for 5 seconds at 50Hz
Tf = 5.0
h = 1.0/50
tsamp = 0:h:Tf
N = length(tsamp)

xhist_mid = zeros(13,N)
xhist_mid[:,1] .= x0

xhist_rk4 = zeros(13,N)
xhist_rk4[:,1] .= x0

# TODO: Implement explicit midpoint with the standard hack of normalizing the quaternion at each step
function normalized_mid_step(xk,uk)
    #q = normalize(xk[4:7])
    #xk[4:7] = q
    xdot = glider_dynamics_q(xk,uk)
    xn = xk + h*glider_dynamics_q(xk+(h/2)*glider_dynamics_q(xk,uk),uk)
    return xn
end

#Implement RK4 with the standard hack of normalizing the quaternion at each step
function normalized_rk4_step(xk,uk)
    
    q = normalize(xk[4:7])
    xk[4:7] = q
    f1 = glider_dynamics_q(xk,uk);
    f2 = glider_dynamics_q(xk+0.5*h*f1,uk);
    f3 = glider_dynamics_q(xk+0.5*h*f2,uk);
    f4 = glider_dynamics_q(xk+h*f3,uk);
    
    xn = xk+(h/6.0)*(f1+2*f2+2*f3+f4);
      
    return xn
end

# TODO: this is where the simulation should happen
for k = 1:(N-1)
    xhist_mid[:,k+1] .= normalized_mid_step(xhist_mid[:,k],u0)
    xhist_rk4[:,k+1] .= normalized_rk4_step(xhist_rk4[:,k],u0)
end

In [ ]:
@testset "first glider sim" begin 
    @test norm(xhist_mid[:,end] - xhist_rk4[:,end]) < 1e-8
    @test xhist_mid[3,1] > xhist_mid[3,end]
    @test isapprox(xhist_mid[2,end],0,atol = 1e-5)
end

Test Summary:    | Pass  Total
first glider sim |    3      3


Test.DefaultTestSet("first glider sim", Any[], 3, false, false)

In [ ]:
#Setup code for the MeshCat visualizer
using RobotDynamics, Rotations
    
struct Glider{R} <: RigidBody{R}
    n::Int
    m::Int
end
RobotDynamics.control_dim(::Glider) = 3

model = Glider{UnitQuaternion}(13,3)

using TrajOptPlots
using MeshCat
using FileIO, MeshIO, Colors, GeometryBasics, CoordinateTransformations
function TrajOptPlots._set_mesh!(vis, model::Glider)
    obj = joinpath(@__DIR__, "piper_scaled.obj")
    robot_obj = FileIO.load(obj)
    mat = MeshPhongMaterial(color=colorant"red")
    setobject!(vis["geom"], robot_obj, mat)
    settransform!(vis["geom"], compose(Translation(0,0,0.07),LinearMap( RotY(pi/2)*RotZ(-pi/2) )))
end

┌ Info: Precompiling RobotDynamics [38ceca67-d8d3-44e8-9852-78a5596522e1]
└ @ Base loading.jl:1423
┌ Info: Precompiling TrajOptPlots [7770976a-8dee-4930-bf39-a1782fd21ce6]
└ @ Base loading.jl:1423


In [ ]:
#Start the visualizer (this can take a while)
vis = Visualizer()
TrajOptPlots.set_mesh!(vis, model)
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /Users/pfisch/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8701))

In [ ]:
#Visualize the trajectory (you'll probably want to zoom out and pan a little bit)
X1 = [SVector{13}(x) for x in eachcol(xhist_rk4)];
visualize!(vis, model, Tf, X1)

## 1C: Make the glider do a loop (20 pts)

In [ ]:
#Let's try something more interesting. We're going to loop the glider.

# TODO: Try some different initial conditions to find some that successfully perform a loop.
#You'll probably want to increase the horizontal velocity and elevator command

# horizontal velocity is v[1] (change this)
# elevator input is u[2] (change this)

v0_loop = v0 + [10;0;0]
x0_loop = [r0; q0; v0_loop; ω0] 
u0_loop = u0 + [0,0.2,0]

xhist_mid = zeros(13,N)
xhist_mid[:,1] .= x0_loop

xhist_rk4 = zeros(13,N)
xhist_rk4[:,1] .= x0_loop

for k = 1:(N-1)
    xhist_mid[:,k+1] .= normalized_mid_step(xhist_mid[:,k],u0_loop) # midpoint here
    xhist_rk4[:,k+1] .= normalized_rk4_step(xhist_rk4[:,k],u0_loop) # rk4 here
end

X1 = [SVector{13}(x) for x in eachcol(xhist_rk4)];
visualize!(vis, model, Tf, X1)

In [ ]:
#Let's also compute a reference solution using DifferentialEquations.jl
using OrdinaryDiffEq

#wrapper function for DifferentialEquations.jl interface 
function f(x,p,t)    
    ẋ = glider_dynamics_q(x,u0_loop)
end

tspan = (0.0,Tf)
prob = ODEProblem(f,x0_loop,tspan)
sol = solve(prob,Tsit5(),reltol=1e-12,abstol=1e-12);

xref = zeros(13,N)
for k = 1:N
    xref[:,k] = sol(tsamp[k])
end

In [ ]:
#Now we're going to implement the glider dynamics using an SE(3) parameterization for the configuration

#The hat map for se(3) that maps [v; ω] into a 4x4 matrix. This will be useful for implementing the SE(3) kinematics
function hat_se3(x)
    ω_hat = hat(x[4:6])
    v_se3 = x[1:3]
    hat_op = [ω_hat v_se3
            0 0 0 0]
    return hat_op
end

function glider_dynamics_se3(T,w,u)
    #Unpack state vector
    r = T[1:3, 4]
    Q = T[1:3, 1:3]
    v = w[1:3] #B frame
    ω = w[4:6] #B frame

    #TODO: implement the se3 dynamics of the glider
    hat_map = hat_se3(w)

    Ṫ = T*hat_map
    F,τ = aero_forces(v,ω,u)
    v̇_se3 = F/m - cross(ω,v) - Q'*[0,0,g]
    ω̇_se3 = J\(τ - cross(ω,J*ω))
    ẇ = [v̇_se3;ω̇_se3]

    return Ṫ, ẇ
end

glider_dynamics_se3 (generic function with 1 method)

In [ ]:
@testset "se3 dynamics" begin 
    
    x = [1;2;3;4;5;6;7]
    H1 = hat_se3(x)
    
    @test size(H1) == size(test_dict["H1"])
    test_vec(H1,"H1",test_dict)
    
    Q = exp(hat([-0.5;2;3]))
    d = [4.5;-6.0;8.9]
    T = [Q d; 0 0 0 1]
    
    w = [10;-.7;.8;.1;-.4;-0.7]
    u = [0.1;0.3;-0.2]
    
    Tdot, wdot = glider_dynamics_se3(T,w,u)
         
    test_vec(Tdot,"Tdot",test_dict)
    test_vec(wdot,"wdot",test_dict)
    
end

Test Summary: | Pass  Total
se3 dynamics  |   39     39


Test.DefaultTestSet("se3 dynamics", Any[Test.DefaultTestSet("element #1 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #2 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #3 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #4 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #5 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #6 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #7 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #8 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #9 of H1", Any[], 1, false, false), Test.DefaultTestSet("element #10 of H1", Any[], 1, false, false)  …  Test.DefaultTestSet("element #13 of Tdot", Any[], 1, false, false), Test.DefaultTestSet("element #14 of Tdot", Any[], 1, false, false), Test.DefaultTestSet("element #15 of Tdot", Any[], 1, false, false), Test.DefaultTestSet("element #16 of Tdot", Any[], 1, false, false), Test.DefaultTestSet("element #1 of

In [ ]:
#Let's implement a lie-group midpoint integrator similar to the one in the first homework, but this time for SE(3) instead of SO(3)

function lie_midpoint_step(Tk,wk,uk)
    
    v_lie_1 = wk[1:3] #B frame
    ω_lie_1 = wk[4:6] #B frame
    Q = Tk[1:3, 1:3]
    F_lie_1,τ_lie_1 = aero_forces(v_lie_1,ω_lie_1,uk)
    v̇_lie_1 = F_lie_1/m - cross(ω_lie_1,v_lie_1) - Q'*[0,0,g]
    ω̇_lie_1 = J\(τ_lie_1 - cross(ω_lie_1,J*ω_lie_1))
    wk1 = wk + 0.5*h*[v̇_lie_1;ω̇_lie_1]
    Tm = Tk*exp(0.5*h*hat_se3(wk))
    Q1 = Tm[1:3, 1:3]
    v_lie_2 = wk1[1:3] #B frame
    ω_lie_2 = wk1[4:6] #B frame
    F_lie_2,τ_lie_2 = aero_forces(v_lie_2,ω_lie_2,uk)
    v̇_lie_2 = F_lie_2/m - cross(ω_lie_2,v_lie_2) - Q1'*[0,0,g]
    ω̇_lie_2 = J\(τ_lie_2 - cross(ω_lie_2,J*ω_lie_2))
    wn = wk + h*[v̇_lie_2;ω̇_lie_2]
    Tn = Tk*exp(h*hat_se3(wk1))
    
    return Tn, wn
end

lie_midpoint_step (generic function with 1 method)

In [ ]:
@testset "se3 lie midpoint" begin 
        
    Q = exp(hat([-0.5;2;3]))
    d = [4.5;-6.0;8.9]
    T = [Q d; 0 0 0 1]
    
    w = [10;-.7;.8;.1;-.4;-0.7]
    u = [0.1;0.3;-0.2]
    
    T2, w2 = lie_midpoint_step(T,w,u)
    display(T2)
       
    test_vec(T2,"T2",test_dict)
    test_vec(w2,"w2",test_dict)
    
end

4×4 Matrix{Float64}:
 -0.849694    0.219189  -0.479558   4.32216
 -0.526168   -0.293567   0.798101  -6.09202
  0.0341528   0.93047    0.364773   8.90004
  0.0         0.0        0.0        1.0

Test Summary:    | Pass  Total
se3 lie midpoint |   22     22


Test.DefaultTestSet("se3 lie midpoint", Any[Test.DefaultTestSet("element #1 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #2 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #3 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #4 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #5 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #6 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #7 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #8 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #9 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #10 of T2", Any[], 1, false, false)  …  Test.DefaultTestSet("element #13 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #14 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #15 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #16 of T2", Any[], 1, false, false), Test.DefaultTestSet("element #1 of w2"

In [ ]:
# TODO: Convert the initial conditions we had before into an SE(3) state
v0_loop = v0 + [10;0;0]
x0_loop = [r0; q0; v0_loop; ω0] 
u0_loop = u0 + [0,0.2,0]
Q0_loop = H'*L(q0)*R(q0)'*H
T0_loop = [Q0_loop r0;
0 0 0 1]
w0_loop = [v0_loop;ω0]

6-element Vector{Float64}:
 20.0
  0.0
  0.5678625279339364
  0.0
  0.0
  0.0

In [ ]:
@testset "se3 initial conditions" begin      
    Q0 = T0_loop[1:3,1:3]    
    x1 = [-1;2;-3]
    x2 = Q0*x1;
    x2_2 = H'*L(q0)*R(q0)'*H*x1
    @test isapprox(x2,x2_2,atol = 1e-10)
end

Test Summary:          | Pass  Total
se3 initial conditions |    1      1


Test.DefaultTestSet("se3 initial conditions", Any[], 1, false, false)

In [ ]:
#TODO: Run the same simulation with the new lie midpoint step integrator

Thist_lie = zeros(4,4,N)
whist_lie = zeros(6,N)
Thist_lie[:,:,1] .= T0_loop
whist_lie[:,1] .= w0_loop
for k = 1:(N-1)
    Tn, wn = lie_midpoint_step(Thist_lie[:,:,k],whist_lie[:,k],u0_loop) # update this line with the lie midpoint step integrator
    Thist_lie[:,:,k+1] .= Tn
    whist_lie[:,k+1] .= wn
end

In [ ]:
#Compare the solutions
err_pos_mid = zeros(N)
err_rot_mid = zeros(N)
err_pos_rk4 = zeros(N)
err_rot_rk4 = zeros(N)
err_pos_lie = zeros(N)
err_rot_lie = zeros(N)
for k = 1:N
    r_ref  = xref[1:3,k]
    q_ref  = xref[4:7,k]
    r_mid  = xhist_mid[1:3,k]
    q_mid  = xhist_mid[4:7,k]
    r_rk4  = xhist_rk4[1:3,k]
    q_rk4  = xhist_rk4[4:7,k]
    r_lie = Thist_lie[1:3, 4,k]
    Q_lie = Thist_lie[1:3, 1:3,k]
    Q_ref = H'*L(q_ref)*R(q_ref)'*H
    
    err_pos_mid[k] = norm(r_mid-r_ref)
    err_pos_rk4[k] = norm(r_rk4-r_ref)
    err_pos_lie[k] = norm(r_lie-r_ref)
    
    
    err_rot_mid[k] = norm(q_mid-q_ref)
    err_rot_rk4[k] = norm(q_rk4-q_ref)
    err_rot_lie[k] = norm(Q_ref-Q_lie)
end

In [ ]:
plot(err_pos_mid,label = "midpoint")
plot!(err_pos_rk4, label = "rk4")
plot!(err_pos_lie, label = "lie midpoint")
xlabel!("timestep")
ylabel!("error")
title!("SE3 Position Errors")

In [ ]:
plot(err_rot_mid,label = "midpoint")
plot!(err_rot_rk4, label = "rk4")
plot!(err_rot_lie, label = "lie midpoint")
xlabel!("timestep")
ylabel!("error")
title!("SE3 Rotation Errors")

In [ ]:
@testset "full sims" begin
    @test abs(maximum(err_pos_mid))<0.1
    @test abs(maximum(err_pos_rk4))<0.1
    @test abs(maximum(err_pos_lie))<0.1
    @test abs(maximum(err_rot_mid))<0.1
    @test abs(maximum(err_rot_rk4))<0.1
    @test abs(maximum(err_rot_lie))<0.1
end

Test Summary: | Pass  Total
full sims     |    6      6


Test.DefaultTestSet("full sims", Any[], 6, false, false)